In [ ]:
from utils.utils import compare_sinr_user_mean,compare_sinr_per_element, analyze_and_plot_best_episode, plot_metrics_vs_user,custom_save_plot_metrics_vs_user
# Define folders
import subprocess
import os
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
channel_size = 15
leo_users = 10
geo_users = 10

In [ ]:
import os

graphs_folder = f"./graphs/graphs_folder_{channel_size}_channels_{leo_users}_leo_user_{geo_users}_geo_user"
data_csv_folder = f"./data_csv_folder/data_csv_folder_{channel_size}_channels_{leo_users}_leo_user_{geo_users}_geo_user"

saved_folder = f"./results_data/saved_data_{channel_size}_channels_{leo_users}_leo_user_{geo_users}_geo_user"

print(f"{channel_size}_channels_{leo_users}_leo_user_{geo_users}_geo_user")

if not os.path.exists(saved_folder) or not os.path.isdir(saved_folder):
    # Folder doesn't exist or is not a directory, skip processing
    print("Folder does not exist. Skipping function execution...")
    

# If folder exists, check if it's empty
if not os.listdir(saved_folder):  # Check if the directory is empty
    print("Folder is empty. Skipping function execution...")
    

print("Folder exists and is not empty. Continuing to run...")



# Create folders if they don't exist
os.makedirs(graphs_folder, exist_ok=True)
os.makedirs(data_csv_folder, exist_ok=True)


In [ ]:
episode_index = 1

In [ ]:
Thrpt = np.load(f'{saved_folder}/Episode_{episode_index}_Thrpt.npy')
SINR = np.load(f'{saved_folder}/Episode_{episode_index}_SINR.npy')
Intf = np.load(f'{saved_folder}/Episode_{episode_index}_Intf.npy')
SE = np.load(f'{saved_folder}/Episode_{episode_index}_SE.npy')  # Newly added

In [ ]:
SINR.shape

In [ ]:
SINR[:,50]

In [ ]:
np.median(SINR)

In [ ]:

# %%


# %%

# # To compare element-wise SINR:
# compare_sinr_per_element(saved_folder)

# # To compare user-mean SINR:
best_episode_index = compare_sinr_user_mean(saved_folder)
# best_episode_index = 10




# %%
# Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')     # (20, 61)
SINR_baseline = np.load(f'{saved_folder}/Baseline_SINR.npy')       # (20, 61)
Intf_baseline = np.load(f'{saved_folder}/Baseline_Intf.npy')       # (20, 61)
SE_baseline = np.load(f'{saved_folder}/Baseline_SE.npy')       # (20, 61)

# Load best episode data
Thrpt_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Thrpt.npy')
SINR_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_SINR.npy')
Intf_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Intf.npy')
SE_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_SE.npy')  # Newly added

# Compute mean across timesteps (axis=1) for each user
mean_thrpt_baseline = np.mean(Thrpt_baseline, axis=1)
mean_thrpt_best = np.mean(Thrpt_best, axis=1)

mean_sinr_baseline = np.mean(SINR_baseline, axis=1)
mean_sinr_best = np.mean(SINR_best, axis=1)

mean_intf_baseline = np.mean(Intf_baseline, axis=1)
mean_intf_best = np.mean(Intf_best, axis=1)



users = np.arange(1, (leo_users+geo_users)+1)  # User indices (1 to 20)




print("mean_sinr_baseline :",mean_sinr_baseline)
print("mean_sinr_best :",mean_sinr_best)

print("mean_thrpt_baseline :",mean_thrpt_baseline)
print("mean_thrpt_best :",mean_thrpt_best)


print("SE_baseline :",SE_baseline.shape)
print("SE_best :",SE_best.shape)




# Transposing to (121, 1)
SE_baseline_transposed = SE_baseline.T
SE_best_transposed = SE_best.T

# Check the result
print("Transposed SE_baseline shape:", SE_baseline_transposed.shape)
print("Transposed SE_best shape:", SE_best_transposed.shape)

# Console output
print(f"✅ Episode Index: {best_episode_index} | Mean Throughput (first 10 users): {np.mean(mean_thrpt_best[:10]):.4f} Mbps")

# Create figure with 2x2 subplots
plt.figure(figsize=(18, 10))

# 1. Throughput
plt.subplot(2, 2, 1)
plt.plot(users, mean_thrpt_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_thrpt_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean Throughput per User')
plt.xlabel('User Index')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)

# 2. SINR
plt.subplot(2, 2, 2)
plt.plot(users, mean_sinr_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_sinr_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean SINR per User')
plt.xlabel('User Index')
plt.ylabel('SINR (dB)')
plt.legend()
plt.grid(True)

# 3. Interference
plt.subplot(2, 2, 3)
plt.plot(users, mean_intf_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_intf_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean Interference per User')
plt.xlabel('User Index')
plt.ylabel('Interference (dB or mW)')
plt.legend()
plt.grid(True)

# 4. Spectral Efficiency (SE)
plt.subplot(2, 2, 4)
plt.plot( np.arange(1, 122), SE_baseline_transposed, label=f'Baseline {best_episode_index}', marker='o',color ='r')
plt.plot( np.arange(1, 122), SE_best_transposed, label=f'SE Episode {best_episode_index}', marker='x', color ='b')
plt.title('Mean Spectral Efficiency per User')
plt.xlabel('User Index')
plt.ylabel('SE (bits/s/Hz)')
plt.legend()
plt.grid(True)

# Save and display
plt.tight_layout()
filename_base = os.path.join(graphs_folder, f'Metrics_vs_User_Episode_{best_episode_index}')
plt.savefig(f'{filename_base}.png', dpi=300)
plt.savefig(f'{filename_base}.pdf', dpi=300)
plt.savefig(f'{filename_base}.eps', dpi=300, format='eps')
print(f"✅ Saved plots to: {filename_base}.{{png, pdf, eps}}")

plt.show()
plt.close()